In [1]:
import numpy as np
from Bodies import MPMImplicitBody1D
from Materials import ElasticMaterial1D
from Grids import MPMGrid1D
import plotly.graph_objects as go

In [2]:
# Setup
dt           = 1e-4         # s
TrussLength  = 0.4          # m
CrossSection = 0.02 * 0.02  # m^2
GridLength   = 1            # m
NoCells      = 4
NoMP         = 10
# Grid
Grid = MPMGrid1D(0,GridLength,NoCells)
# Material
Emod = 1e6         # Pa = N/m^2
rho  = 1500        # kg/m^3
mue  = 0           # Pa s
Mate = ElasticMaterial1D(Emod, mue, rho)
# Body
x_P = np.linspace(1e-5, TrussLength, NoMP)
V_P = np.ones(NoMP) * CrossSection * TrussLength/NoMP
Body = MPMImplicitBody1D(np.array([x_P, V_P]).T, Mate)

In [3]:
# Time step estimation
cp = Grid.dx / np.sqrt(Emod/rho)
# Target elongation
dl = (9.81 * rho)/(2.0 * Emod) * TrussLength * TrussLength

In [4]:
# Initialization
t    = 0.0                                  # Global Time
# Grid
Grid.u  = np.zeros(Grid.NoNodes)            # Vector of Nodal Unknowns/Displacements
Grid.R  = np.zeros(Grid.NoNodes)            # Vector of Nodal Right hand side
Grid.K  = np.zeros((Grid.NoNodes,Grid.NoNodes))            # Grid Tangent Matrix

# Body
Body.m  = Body.V * Body.Mate.rho            # Vector of Particle Masses
Body.v  = np.zeros(Body.NoMP)               # Vector of Particle Velocities
Body.mv = Body.m * Body.v                   # Vector of Partilce Momentum
Body.a  = np.zeros(Body.NoMP)               # Vector of Partilce Accelerations
Body.F  = np.zeros(Body.NoMP)               # Vector of Partilce Deformation Gradients (stress free reference config)
Body.S  = np.zeros(Body.NoMP)               # Vector of Particle Stresses (stress free reference config)
Body.b  = np.ones(Body.NoMP) * 9.81         # Vector of Particle Accelerations due to gravity m/s^2

In [24]:
# Time Step:
# Shift Particle Data
Body.x_n = np.array(Body.X, copy=True)
Body.v_n = np.array(Body.v, copy=True)
Body.a_n = np.array(Body.a, copy=True)
Body.F_n = np.array(Body.F, copy=True)
Body.V_n = np.array(Body.V, copy=True)
Body.S_n = np.array(Body.S, copy=True)
# Reset Solution Data
Grid.u  = np.zeros(Grid.NoNodes)

In [26]:
# Clear System Matrices
Grid.R  = np.zeros(Grid.NoNodes)                # reset right hand side
Grid.K  = np.zeros((Grid.NoNodes,Grid.NoNodes)) # reset stiffness matix
# Assemble sytsem matrices
for P in range(NoMP):
    N_I = Grid.GridNodes(Body.x_n[P])
    Body.SKR(P, N_I, Grid, dt)
# Form Linear system
constrainerd_dof = [0]
inactive_dof = np.where(Grid.R==0)[0]
I_full = np.diag(np.ones(Grid.NoNodes))
I_red = np.delete(I_full, np.concatenate([constrainerd_dof,inactive_dof]), 0)
# reduce rhs
RHS = I_red.dot(Grid.R)
print(RHS)
# reduce lhs
LHS = I_red.dot((Grid.K).dot(I_red.T))
print(LHS)
# solve system
print('Residual: ',np.sqrt(RHS.dot(RHS)))
Kinv = np.linalg.inv(LHS)
du = - Kinv.dot(RHS)
print(du)
# resolve full displacement vector
Grid.u += du.dot(I_red)

[-6.9388939e-17]
[[706.30892817]]
Residual:  6.938893903907228e-17
[9.82416281e-20]


In [27]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = Body.X,
        y = np.zeros(Body.NoMP),
        mode='markers',
        name='MaterialPoints',
        marker_size = 10
    )
)
fig.add_trace(
    go.Scatter(
        x = Grid.GridNodeX(range(Grid.NoNodes)),
        y = np.zeros(Grid.NoNodes),
        mode='markers+lines',
        name='Grid'
    )
)
fig.show()

In [5]:
def ImplicitTimeStep(dt):
    # Shift Particle Data
    Body.x_n = np.array(Body.X, copy=True)
    Body.v_n = np.array(Body.v, copy=True)
    Body.a_n = np.array(Body.a, copy=True)
    Body.F_n = np.array(Body.F, copy=True)
    Body.V_n = np.array(Body.V, copy=True)
    Body.S_n = np.array(Body.S, copy=True)

    # Reset Solution Data
    Grid.u  = np.zeros(Grid.NoNodes)

    # Newton Loop
    for Iter in range(5):

        # Clear System Matrices
        Grid.R  = np.zeros(Grid.NoNodes)                # reset right hand side
        Grid.K  = np.zeros((Grid.NoNodes,Grid.NoNodes)) # reset stiffness matix

        # Assemble sytsem matrices
        for P in range(NoMP):
            N_I = Grid.GridNodes(Body.x_n[P])
            Body.SKR(P, N_I, Grid, dt)

        # Form Linear system
        constrainerd_dof = [0]
        inactive_dof = np.where(Grid.R==0)[0]
        I_full = np.diag(np.ones(Grid.NoNodes))
        I_red = np.delete(I_full, np.concatenate([constrainerd_dof,inactive_dof]), 0)

        # reduce rhs
        RHS = I_red.dot(Grid.R)
        print(RHS)

        # reduce lhs
        LHS = I_red.dot((Grid.K).dot(I_red.T))
        print(LHS)

        # check for convergency
        residual = np.sqrt(RHS.dot(RHS))
        print('Residual: ',residual)
        if (residual) < 1e-8:
            break

        # solve system
        Kinv = np.linalg.inv(LHS)
        du = - Kinv.dot(RHS)

        # resolve full displacement vector
        Grid.u += du.dot(I_red)

In [6]:
Time, NoSteps = 0, 10000
x_t = np.zeros((NoSteps,4))
for t in range(NoSteps):
    Time += dt
    ImplicitTimeStep(dt)
    x_t[t] = np.array([Time, Body.X[-1], Body.StrainEnergy(), Body.KineticEnergy()])

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_t[:,0], y=x_t[:,1],
    line=dict(color='royalblue', width=2))
    )
fig.update_layout(
    title='Truss-Lenght over Time',
    xaxis_title='time in s',
    yaxis_title='length in m'
    )
fig.show()

8.84038443]
[[35274932.19264506  7019069.22203441]
 [ 7019069.22203441  5787948.00196112]]
Residual:  141.87162275628143
[2.96984659e-15 7.50094431e-15]
[[35274932.19264506  7019069.22203441]
 [ 7019069.22203441  5787948.00196112]]
Residual:  8.067475089325848e-15
[129.14283137  47.10644323]
[[35275259.28546301  7018916.39972533]
 [ 7018916.39972533  5787557.30573563]]
Residual:  137.465951744995
[-5.27355937e-15  1.79023463e-15]
[[35275259.28546301  7018916.39972533]
 [ 7018916.39972533  5787557.30573563]]
Residual:  5.569144316446332e-15
[125.07819967  45.36373388]
[[35275573.47706421  7018767.98178426]
 [ 7018767.98178426  5787180.47939075]]
Residual:  133.05045803928104
[-3.11417558e-14  2.70616862e-15]
[[35275573.47706421  7018767.98178426]
 [ 7018767.98178426  5787180.47939075]]
Residual:  3.125911555782609e-14
[121.00574067  43.61282063]
[[35275874.67215343  7018623.97238282]
 [ 7018623.97238282  5786817.59307297]]
Residual:  128.62529843799211
[-1.15740750e-14  7.57033325e-15]


In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_t[:,0], y=x_t[:,2],
    mode='lines',
    name='Strain Energy'
    ))
fig.add_trace(go.Scatter(
    x=x_t[:,0], y=x_t[:,3],
    mode='lines',
    name='Kinetic Energy'
    ))
fig.add_trace(go.Scatter(
    x=x_t[:,0], y=x_t[:,2]+x_t[:,3],
    mode='lines',
    name='Total Energy'
    ))
fig.update_layout(
    title='Energy over Time',
    xaxis_title='time in s',
    yaxis_title='energy'
    )
fig.show()

In [7]:
print(dl)
print(x_t[-1,1]-TrussLength)

0.00029430000000000005
0.0002972376200958693


In [8]:
# NoMP, NoCells, u
data = np.array([
    [50, 1, 0.0029419606870446924],
    [50, 3, 0.0009809975466645027],
    [50, 6, 0.0006679994576182235],
    [50, 11, 0.0006068314007743048],
    [50, 21, 0.0006026618362144076],
    [50, 51, 0.0006000933024597754],
    [50, 81, 0.0006215417166090531],
    [50, 111, 0.000626491093039]
])
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data[:,1], y=data[:,2],
    line=dict(color='royalblue', width=2))
    )
fig.update_layout(
    title='Convergency for Numbers of Cells',
    xaxis_title='Number of Cells',
    yaxis_title='Final Elongation'
    )
fig.show()

In [12]:
# NoMP, NoCells, u
data = np.array([
    [5, 11, 0.0007528157123913581],
    [10, 11, 0.0005703161898724107],
    [20, 11, 0.0006173523716542229],
    [50, 11, 0.0006068314007743048],
    [100, 11, 0.0006173464774665571]
    ])
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data[:,0], y=data[:,2],
    line=dict(color='royalblue', width=2))
    )
fig.update_layout(
    title='Convergency for Numbers of MaterialPoints',
    xaxis_title='Number of MP',
    yaxis_title='Final Elongation'
    )
fig.show()